**Phase One of the Project: Data Extraction**  - Jenifer Ramos Rios - Code: 22500237

Data base connection

In [1]:
# Librerias
import yaml
import psycopg2 
from psycopg2 import sql
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, DateTime, Float
import pandas as pd
import os
import re

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file: # Abre el archivo en modo lectura ("r")
        return yaml.safe_load(file) # Carga y devuelve el contenido del archivo YAML

In [3]:
config = load_config()
db_config = config["database"]

# Load credentials, se accede a los valores de la clave "database" del archivo config.yaml
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

In [4]:
#Creating the database 
db_name = "ETL_Project"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close() #Close the database connection

La base de datos 'ETL_Project' ya existe.


File Reading for Review and Table Creation

In [5]:
# Specify the directory where the files are located
first_course_path = "D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso"

#List to store file names
first_course_file_list = []

# Iterate over the files in the directory
for filename in os.listdir(first_course_path):
    # Check if the file has the .xlsx extension
    if filename.endswith(".xlsx"):
        # Add the file to the list
        first_course_file_list.append(os.path.join(first_course_path, filename))

# Print the list of files
print("Excel (xlsx) files found:", first_course_file_list)

Excel (xlsx) files found: ['D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2017_1RECURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2018_1ERCURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2019_1ERCURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2020_1ERCURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2021_1ERCURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2022_1ERCURSO.xlsx', 'D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\\2023_1ERCURSO.xlsx']


In [6]:
table_names = []

# Crear una conexión a la base de datos ETL_Project
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Leer los archivos en dataframes y cargarlos en la base de datos
for file in first_course_file_list:
    # Leer el archivo en un dataframe
    df = pd.read_excel(file, header=None)
    
    # Hacer que la primera fila sea el encabezado
    df.columns = df.iloc[0]
    df = df[1:]

    # Cambiar los encabezados a mayúsculas y eliminar tildes
    df.columns = df.columns.str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Obtener el nombre del archivo sin la extensión
    table_name = "tabla_" + os.path.splitext(os.path.basename(file))[0]
    table_names.append(table_name)

    # Cargar el dataframe en la base de datos
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Datos del archivo {file} cargados en la tabla '{table_name}' de la base de datos.")

Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2017_1RECURSO.xlsx cargados en la tabla 'tabla_2017_1RECURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2018_1ERCURSO.xlsx cargados en la tabla 'tabla_2018_1ERCURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2019_1ERCURSO.xlsx cargados en la tabla 'tabla_2019_1ERCURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2020_1ERCURSO.xlsx cargados en la tabla 'tabla_2020_1ERCURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2021_1ERCURSO.xlsx cargados en la tabla 'tabla_2021_1ERCURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2022_1ERCURSO.xlsx cargados en la tabla 'tabla_2022_1ERCURSO' de la base de datos.
Datos del archivo D:/JENN/Maestria IACD/ETL/ETL-Class-Project/1erCurso\2023_

In [7]:
# leer las tablas de la DB en dataframes y mostrar los encabezados de cada tabla
dataframes = []
for tablabd in table_names:
    with engine.connect() as conn:
        df = pd.read_sql(f'SELECT * FROM "{tablabd}"', conn)
        dataframes.append(df)
        print(f"Tabla '{tablabd}':")
        pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
        print(df)

Tabla 'tabla_2017_1RECURSO':
       CODIGO DE \nLA INSTITUCION  IES PADRE  \
0                            1101       1101   
1                            1101       1101   
2                            1101       1101   
3                            1101       1101   
4                            1101       1101   
...                           ...        ...   
44250                        9922       9922   
44251                        9922       9922   
44252                        9922       9922   
44253                        9922       9922   
44254                        9922       9922   

             INSTITUCION DE EDUCACION SUPERIOR (IES) PRINCIPAL\n O\nSECCIONAL  \
0                   UNIVERSIDAD NACIONAL DE COLOMBIA                PRINCIPAL   
1                   UNIVERSIDAD NACIONAL DE COLOMBIA                PRINCIPAL   
2                   UNIVERSIDAD NACIONAL DE COLOMBIA                PRINCIPAL   
3                   UNIVERSIDAD NACIONAL DE COLOMBIA                PR

In [8]:
# Palabras clave para filtrar las columnas
keywords = ['SNIES', 'SEXO', 'AÑO', 'ANO', 'PRIMER']

# Iterar sobre cada DataFrame y realizar las transformaciones necesarias
filtered_dataframes = []
for i, df in enumerate(dataframes):
    
    # Añadir la descripción de sexo
    df['SEXO'] = df['ID SEXO'].apply(lambda x: 'Masculino' if x == 1 else 'Femenino' if x == 2 else 'Desconocido')
    
    # Filtrar las columnas que contienen las palabras clave, esto genera una copia con las columnas seleccionadas.
    filtered_columns = df.filter(regex='|'.join(keywords)).columns
    filtered_df = df[filtered_columns]
        
    # Renombrar las columnas específicas
    filtered_df = filtered_df.rename(columns={
        'CODIGO \nSNIES DEL\nPROGRAMA': 'CODIGO SNIES DEL PROGRAMA',
        'PRIMER CURSO 2017': 'PRIMER CURSO',
        'PRIMER CURSO 2018': 'PRIMER CURSO',
        'PRIMER CURSO 2019': 'PRIMER CURSO',
        'ANO': 'AÑO'
    })
    
    # Actualizar el dataframe en la lista de dataframes
    dataframes[i] = filtered_df

    #Agregar el DataFrame filtrado a la lista de DataFrames filtrados
    filtered_dataframes.append(filtered_df)

# Mostrar los encabezados de los DataFrames actualizados
for i, df in enumerate(dataframes):
    print(df.head())


   CODIGO SNIES DEL PROGRAMA  ID SEXO       SEXO   AÑO  PRIMER CURSO
0                          1        1  Masculino  2017            74
1                          1        1  Masculino  2017            62
2                          1        2   Femenino  2017            30
3                          1        2   Femenino  2017            40
4                          2        1  Masculino  2017            26
   CODIGO SNIES DEL PROGRAMA  ID SEXO       SEXO   AÑO  PRIMER CURSO
0                          1        1  Masculino  2018            72
1                          1        1  Masculino  2018            54
2                          1        2   Femenino  2018            30
3                          1        2   Femenino  2018            25
4                          2        1  Masculino  2018            26
  CODIGO SNIES DEL PROGRAMA  ID SEXO   AÑO  PRIMER CURSO       SEXO
0                         1        1  2019            64  Masculino
1                         1        1

In [9]:
# Concatenar todos los DataFrames filtrados en uno solo
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

# Seleccionar las columnas necesarias
combined_df = combined_df[['CODIGO SNIES DEL PROGRAMA', 'SEXO', 'AÑO', 'PRIMER CURSO']]

# Función para limpiar caracteres especiales, comas y puntos
def limpiar_caracteres(texto):
    if isinstance(texto, str):
        return re.sub(r'[^\d]', '', texto)
    return texto

# Aplicar la función de limpieza a las columnas pertinentes
combined_df['CODIGO SNIES DEL PROGRAMA'] = combined_df['CODIGO SNIES DEL PROGRAMA'].apply(limpiar_caracteres)
combined_df['AÑO'] = combined_df['AÑO'].apply(limpiar_caracteres)
combined_df['PRIMER CURSO'] = combined_df['PRIMER CURSO'].apply(limpiar_caracteres)

# Convertir las columnas a numéricas, forzando errores a NaN
combined_df['CODIGO SNIES DEL PROGRAMA'] = pd.to_numeric(combined_df['CODIGO SNIES DEL PROGRAMA'], errors='coerce')
combined_df['AÑO'] = pd.to_numeric(combined_df['AÑO'], errors='coerce')
combined_df['PRIMER CURSO'] = pd.to_numeric(combined_df['PRIMER CURSO'], errors='coerce')

# Eliminar filas con valores NaN generados por errores de conversión y contar las filas eliminadas
initial_row_count = combined_df.shape[0]
combined_df.dropna(subset=['CODIGO SNIES DEL PROGRAMA', 'AÑO', 'PRIMER CURSO'], inplace=True)
final_row_count = combined_df.shape[0]
print(f"Filas totales: {final_row_count}")
rows_dropped = initial_row_count - final_row_count
print(f"Filas eliminadas: {rows_dropped}")

# Convertir tipos de datos correctamente
combined_df = combined_df.astype({
    'CODIGO SNIES DEL PROGRAMA': 'int64',
    'AÑO': 'int64',
    'PRIMER CURSO': 'int64',
    'SEXO': 'string'
})

Filas totales: 269187
Filas eliminadas: 49272


In [10]:
# Crear una conexión a la base de datos ETL_Project
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Nombre de la nueva tabla
combined_table_name = 'PrimerCurso2017-2023'

# Definir el tipo de dato de cada columna
dtype = {
    'CODIGO SNIES DEL PROGRAMA': Integer(),
    'SEXO': String(),
    'AÑO': Integer(),
    'PRIMER CURSO': Integer()
}

# Cargar el DataFrame combinado en la base de datos
combined_df.to_sql(combined_table_name, engine, if_exists='replace', index=False, dtype=dtype)
print(f"Datos combinados cargados en la tabla '{combined_table_name}' de la base de datos.")

# Mostrar los primeros registros de la tabla combinada
with engine.connect() as conn:
    result_df = pd.read_sql(f'SELECT * FROM "{combined_table_name}" LIMIT 5', conn)
    print(result_df)

Datos combinados cargados en la tabla 'PrimerCurso2017-2023' de la base de datos.
   CODIGO SNIES DEL PROGRAMA       SEXO   AÑO  PRIMER CURSO
0                          1  Masculino  2017            74
1                          1  Masculino  2017            62
2                          1   Femenino  2017            30
3                          1   Femenino  2017            40
4                          2  Masculino  2017            26


In [11]:
# Crear una conexión a la base de datos ETL_Project
with engine.connect() as conn:
    result = conn.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    tables = result.fetchall()
    table_list = [table[0] for table in tables]
    print("Tablas en la base de datos ETL_Project:", table_list)

Tablas en la base de datos ETL_Project: ['tabla_2022_1ERCURSO', 'tabla_2023_1ERCURSO', 'PrimerCurso2017-2023', 'Inscritos2017-2023', 'Table_Matricula', 'tabla_2017_1RECURSO', 'tabla_2018_1ERCURSO', 'tabla_2019_1ERCURSO', 'tabla_2020_1ERCURSO', 'tabla_2021_1ERCURSO']


In [12]:
# Crear una conexión a la base de datos ETL_Project
with engine.connect() as conn:
    inscritos_df = pd.read_sql(f'SELECT * FROM "{'PrimerCurso2017-2023'}"', conn)
    print(inscritos_df.head())
    # Mostrar todos los valores de la columna 'CODIGO SNIES DEL PROGRAMA'
    print(inscritos_df['CODIGO SNIES DEL PROGRAMA'])


   CODIGO SNIES DEL PROGRAMA       SEXO   AÑO  PRIMER CURSO
0                          1  Masculino  2017            74
1                          1  Masculino  2017            62
2                          1   Femenino  2017            30
3                          1   Femenino  2017            40
4                          2  Masculino  2017            26
0              1
1              1
2              1
3              1
4              2
           ...  
269182    109846
269183    110347
269184    110347
269185    110347
269186    110347
Name: CODIGO SNIES DEL PROGRAMA, Length: 269187, dtype: int64
